# Keyword Extraction: Korpusvergleich

Schlüsselwörter können nicht nur für einzelne Dokumente identifiziert werden, sondern auch für ganze Teilkorpora. Das Verfahren ist identisch, nur dass charakteristische Wörter für eine Gruppe an Dokumenten im Vergleich zum Gesamtkorpus gefunden werden. Dies erlaubt interessante Vergleiche, z.B. nach Autor, Jahr, oder Genre.

Der Anfang ist identisch mit der bisherigen Keyword Extraction.

In [1]:
from gensim.corpora.textcorpus import TextCorpus
from textblob_de import TextBlobDE as TextBlob
import pandas as pd

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def get_texts(self):
        with self.getstream() as csvfile:  # Öffnet die CSV-Datei
            table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')  # Liest die CSV-Datei
            for text in table['text']:  # Verarbeite die einzelnen Texte aus der Spalte 'text'
                blob = TextBlob(text)
                yield blob.words

Zunächst wird das IDF-Referenzmodell für das gesamte Korpus erstellt:

In [2]:
from gensim.models import TfidfModel

corpus = CSVCorpus("../Daten/Reden.csv")
tf_idf = TfidfModel(corpus)

Der Ansatz beim Vergleich eines Teilkorpus ist, das Teilkorpus als ein einziges Dokument aufzufassen. Dazu werden die Termhäufigkeiten der Dokumente des Teilkorpus einfach aufaddiert. Dies ist leichter, wenn das Korpus als Matrix bzw. DataFrame vorliegt, weshalb das Gensim-Korpus zunächst in die entsprechende Form gebracht wird:

In [3]:
from gensim.matutils import corpus2csc

corpus_matrix_sparse = corpus2csc(corpus)
corpus_matrix_sparse = corpus_matrix_sparse.transpose()
corpus_matrix = corpus_matrix_sparse.todense()
df = pd.DataFrame(corpus_matrix)
df.shape

(793, 65820)

Für die Erstellung der Subkorpora wird auf die Metadaten zugegriffen, die in der CSV-Datei gespeichert sind. Für den Vergleich von Rednern steht keine explizite Spalte zur Verfügung, daher wird näherungsweise nach dem Vorkommen des Namens im Titel geguckt:

In [4]:
data = pd.read_csv("../Daten/Reden.csv", parse_dates=['date'], encoding='utf-8')

In [5]:
has_mg = data['title'].apply(lambda title: 'Grütters' in title)
df_author = df[has_mg]
df_author.shape

(95, 65820)

In [6]:
subcorpus = df_author.sum()
subcorpus = subcorpus.astype(int)
subcorpus = subcorpus[subcorpus > 0]
subcorpus.head()

0    19
2    59
3    23
4    45
6    19
dtype: int64

Für Gensim muss das Subkorpus nun wieder als ein Dokument vorliegen, in dem die Worthäufigkeiten wieder in der Form `[(id, num), (id, num), …]` angegeben sind. Dies geht mit folgendem Befehl:

In [7]:
doc = list(subcorpus.items())
doc[0:5]

[(0, 19), (2, 59), (3, 23), (4, 45), (6, 19)]

Damit lassen sich nun die Schlüsselwörter für das gesamte Teilkorpus berechnen:

In [8]:
doc_tf_idf = tf_idf[doc]

In [9]:
def keywords(corpus, doc_tf_idf):
    return [(corpus.dictionary[term], value)
            for term, value
            in sorted(doc_tf_idf, key=lambda x: x[1], reverse=True)]

In [10]:
keywords(corpus, doc_tf_idf)[0:10]

[('Kunst', 0.16511966593032212),
 ('Kultur', 0.15117397698367702),
 ('Künstler', 0.12729340868694483),
 ('Museen', 0.096764694393125045),
 ('Kirche', 0.093155938441759248),
 ('digitalen', 0.084813307839137778),
 ('kulturellen', 0.080118824389458568),
 ('Freiheit', 0.074491567587995661),
 ('Euch', 0.070873320965894951),
 ('kulturelle', 0.070373627229765479)]

Um dies nun vergleihend für andere Redner durchzuführen, können die Schritte auch in einer Funktion zusammengefasst werden.

In [11]:
def subcorpus_for_author(corpus, metadata, author):
    has_author = data['title'].apply(lambda title: author in title)
    corpus_author = corpus[has_author]
    subcorpus = corpus_author.sum()
    subcorpus = subcorpus.astype(int)
    subcorpus = subcorpus[subcorpus > 0]
    doc = list(subcorpus.items())
    return doc

In [12]:
doc = subcorpus_for_author(df, data, 'Neumann')
doc_tf_idf = tf_idf[doc]
keywords(corpus, doc_tf_idf)[0:10]

[('Kultur', 0.1571502414512424),
 ('Film', 0.11341300996198402),
 ('Kunst', 0.11339975406015394),
 ('Ausstellung', 0.11042362116999693),
 ('Welle', 0.093718845440885129),
 ('Stiftung', 0.093369458934023306),
 ('Filme', 0.083104206622181695),
 ('Museen', 0.077426726136711962),
 ('Museum', 0.07524065855182438),
 ('Haus', 0.071941142286260154)]